In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import os
from PIL import Image
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
#from linformer import Linformer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%pip install einops
%pip install wandb

In [ ]:
%cd drive/MyDrive/Colab Notebooks/ViT

[Errno 2] No such file or directory: 'drive/MyDrive/Colab Notebooks/ViT'
/content/drive/MyDrive/Colab Notebooks/ViT


In [ ]:
from model.vit import ViT
from dataloader import set_transforms
from utils.utils import seed_everything

seed_everything(42)

In [ ]:
import torch.backends.cudnn as cudnn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = torch.cuda.is_available()
cudnn.benchmark = True
print('Use CUDA:', use_cuda)

Use CUDA: True


In [ ]:
config={
    "epochs": 20, 
    "batch_size": 64,
    "lr" : 3e-5,
    "gamma" : 0.7,
    "image_size" : 32,
    "patch_size" : 16,
    "num_classes" : 10,
    "dim" : 128,
    "depth" : 12,
    "heads" : 8,
    "mlp_dim" : 1024,
    "channels" : 3,
    "dropout" : 0.,
    "emb_dropout": 0.
    }

# Training settings
import wandb
wandb.init(config=config, project="Cifar10_ViT", entity='NoguNogu',name='Vanila')

Epoch,20
loss,1.49023
acc,0.46395
val_loss,1.51573
val_acc,0.4581
_runtime,2184
_timestamp,1620287387
_step,19


Epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,█▇▅▅▄▄▃▃▃▃▂▂▂▂▂▂▁▁▁▁
acc,▁▃▄▄▅▅▆▆▆▆▇▇▇▇▇▇████
val_loss,█▇▆▅▄▄▄▃▃▃▂▂▂▂▂▁▁▁▁▁
val_acc,▁▂▃▄▅▄▅▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_timestamp,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
_step,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██


In [ ]:
trainloader, testloader = set_transforms(
    config['image_size'],
    config['batch_size'],
    name='Cifar10')._set_transforms()

In [ ]:
model = ViT(
    dim = config['dim'],
    image_size = config['image_size'],
    patch_size = config['patch_size'],
    num_classes= config['num_classes'],
    channels= config['channels'],
    depth= config['depth'],
    heads= config['heads'],
    mlp_dim = config['mlp_dim']
    ).to(device)

In [ ]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=config['lr'])
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=config['gamma'])

In [ ]:
# 3. Log gradients and model parameters
wandb.watch(model, log_freq=100)
for epoch in range(config['epochs']):
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(trainloader):
        data = data.to(device)
        label = label.to(device)

        output = model(data)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(trainloader)
        epoch_loss += loss / len(trainloader)

    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in testloader:
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(testloader)
            epoch_val_loss += val_loss / len(testloader)

    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )

    # 4. Log metrics to visualize performance
    wandb.log({
        "Epoch": epoch+1,
        "loss": epoch_loss,
        "acc": epoch_accuracy,
        "val_loss" : epoch_val_loss,
        "val_acc": epoch_val_accuracy
        })


Epoch : 1 - loss : 2.0699 - acc: 0.2227 - val_loss : 1.9753 - val_acc: 0.2770




Epoch : 2 - loss : 1.9506 - acc: 0.2795 - val_loss : 1.8940 - val_acc: 0.2976




Epoch : 3 - loss : 1.8591 - acc: 0.3179 - val_loss : 1.8126 - val_acc: 0.3365




Epoch : 4 - loss : 1.7974 - acc: 0.3413 - val_loss : 1.7743 - val_acc: 0.3491




Epoch : 5 - loss : 1.7526 - acc: 0.3621 - val_loss : 1.7310 - val_acc: 0.3704



...


Epoch : 16 - loss : 1.5369 - acc: 0.4453 - val_loss : 1.5449 - val_acc: 0.4407




Epoch : 17 - loss : 1.5251 - acc: 0.4507 - val_loss : 1.5448 - val_acc: 0.4461




Epoch : 18 - loss : 1.5142 - acc: 0.4566 - val_loss : 1.5294 - val_acc: 0.4488




Epoch : 19 - loss : 1.4972 - acc: 0.4604 - val_loss : 1.5159 - val_acc: 0.4601




Epoch : 20 - loss : 1.4902 - acc: 0.4640 - val_loss : 1.5157 - val_acc: 0.4581



In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Fri Apr 23 06:39:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    44W / 250W |   2451MiB / 16280MiB |     25%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------